# Hidden shift algorithm


[Bravyi and Gosset](https://arxiv.org/abs/1601.07601v3) and [Bravyi *et. al*](https://arxiv.org/abs/1808.00128v2) state that the hidden shift problem is particularly suited to benchmark their quantum circuit simulator because:

1. the output is deterministic;
and
2. the $T$ count of the algorithm is easily controlled by choosing a suitable bent function.

Following the description presented in the Appendix F of [[Bravyi and Gosset]](https://arxiv.org/abs/1601.07601v3), I will implement the hidden shift algorithm here as well.

---

This notebook was used to generate the following samples of the hidden-shift algorithm with the goal of benchmarking our code:

* $t=14$:
    1. $n=10$;
    2. $n=14$;
    3. $n=18$;
    4. $n=22$;
    5. $n=28$;
    6. $n=32$;
    

* $t=28$:
    1. $n=14$;
    2. $n=18$;
    3. $n=22$;
    4. $n=28$;
    5. $n=32$;
    6. $n=36$.

In [ ]:
import qiskit
import random
import os

from qiskit.providers.aer import StatevectorSimulator, QasmSimulator
from qiskit.tools.visualization import plot_histogram

In [ ]:
from HSA import HSA_gen

#### <span style=color:green>Sample generation!</span>
    

In [ ]:
def sample_gen(nr_samples, n, n_ccz, s, s_string):
    nr_samples = 100
    
    for circuit in range(nr_samples):

        qc = HSA_gen (n, n_ccz, g, s)
        qc.qasm(
            formatted=False,
            filename=
            f'tcount{n_ccz*14}--n{n}/s{s_string}/_samples/HSA{circuit}.qasm')

In [ ]:
def Toff_decomp(nr_samples, n, n_ccz, s_string):
    for circuit in range(nr_samples):
        with open(
                f'tcount{n_ccz*14}--n{n}/s{s_string}/_samples/HSA{circuit}.qasm'
        ) as file_object:
            file_lines = file_object.readlines()

            clines = file_lines.copy()
            const = 0
            for line in clines:
                if line.startswith('ccx'):
                    index = clines.index(line) + const
                    a = line.partition('[')
                    b = a[2].partition(']')
                    ctrl_qubit1 = int(b[0])

                    c = b[2].partition('[')
                    d = c[2].partition(']')
                    ctrl_qubit2 = int(d[0])

                    e = d[2].partition('[')
                    f = e[2].partition(']')
                    target_qubit = int(f[0])

                    file_lines.remove(line)
                    # Toffoli decomposition:

                    # (1) controlled-not gate between qubits 1 and 2:
                    file_lines.insert(
                        index, f'cx qr[{ctrl_qubit1}],qr[{ctrl_qubit2}];\n')
                    # (2) T gate on qubit 1:
                    file_lines.insert(index, f't qr[{ctrl_qubit1}];\n')
                    # (3) T^dagger on qubit 2:
                    file_lines.insert(index, f's qr[{ctrl_qubit2}];\n')
                    file_lines.insert(index, f's qr[{ctrl_qubit2}];\n')
                    file_lines.insert(index, f's qr[{ctrl_qubit2}];\n')
                    file_lines.insert(index, f't qr[{ctrl_qubit2}];\n')
                    # (4) controlled-not gate between qubits 1 and 2:
                    file_lines.insert(
                        index, f'cx qr[{ctrl_qubit1}],qr[{ctrl_qubit2}];\n')
                    # (5) H gate on qubit 3:
                    file_lines.insert(index, f'h qr[{target_qubit}];\n')
                    # (6) T gates on qubits 2 and 3:
                    file_lines.insert(index, f't qr[{ctrl_qubit2}];\n')
                    file_lines.insert(index, f't qr[{target_qubit}];\n')
                    # (7) controlled-not gate between qubits 1 and 3:
                    file_lines.insert(
                        index, f'cx qr[{ctrl_qubit1}],qr[{target_qubit}];\n')
                    # (8) T^dagger on qubit 3:
                    file_lines.insert(index, f's qr[{target_qubit}];\n')
                    file_lines.insert(index, f's qr[{target_qubit}];\n')
                    file_lines.insert(index, f's qr[{target_qubit}];\n')
                    file_lines.insert(index, f't qr[{target_qubit}];\n')
                    # (9) controlled-not gate between qubits 2 and 3:
                    file_lines.insert(
                        index, f'cx qr[{ctrl_qubit2}],qr[{target_qubit}];\n')
                    # (10) T gate on qubit 3:
                    file_lines.insert(index, f't qr[{target_qubit}];\n')
                    # (11) controlled-not gate between qubits 1 and 3:
                    file_lines.insert(
                        index, f'cx qr[{ctrl_qubit1}],qr[{target_qubit}];\n')
                    # (12) T^dagger on qubit 3:
                    file_lines.insert(index, f's qr[{target_qubit}];\n')
                    file_lines.insert(index, f's qr[{target_qubit}];\n')
                    file_lines.insert(index, f's qr[{target_qubit}];\n')
                    file_lines.insert(index, f't qr[{target_qubit}];\n')
                    # (13) controlled-not gate between qubits 2 and 3:
                    file_lines.insert(
                        index, f'cx qr[{ctrl_qubit2}],qr[{target_qubit}];\n')
                    # (14) H gate on qubit 3:
                    file_lines.insert(index, f'h qr[{target_qubit}];\n')
                    # This concludes the Toffoli decomposition
                    const += 23

        with open(
                f'tcount{n_ccz*14}--n{n}/s{s_string}/_samples/HSA{circuit}-input.qasm',
                'w') as file_object:
            for line in file_lines:
                file_object.write(line)

In [ ]:
def create_path (n_ccz, n, s_string):
    # verify whether the required folders exist
    path = f'tcount{n_ccz*14}--n{n}'
    if (not os.path.isdir(path)):
        # create
        os.mkdir(path) 
    path = path + f'/s{s_string}'
    if (not os.path.isdir(path)):
        # create
        os.mkdir(path) 
    path = path + f'/_samples'
    if (not os.path.isdir(path)):
        # create
        os.mkdir(path)     


## $T$-count: $t=14$

### $n=10$


In [ ]:
n = 10

In [ ]:
# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

print (s, type(s))

In [ ]:
n_ccz = 1 # implies t_count = 14
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

#### Remarks:

The generated quantum circuits will have the following properties:

* $n=10$ qubits,
* $800$ gates from the $\{ Z, \, CZ \}$ set (since $g=200$),
* $14$ $T$ gates,
and
* $w=10$ measurements.

The output of the computation is **deterministic** and corresponds to the string $s \in \mathbb{F}^{10}_2$ generated at random.

We want to export the randomly generated quantum circuits to a `.qasm` file and use them as input for our compilation code.

Given the way the compilation code is written, we will require a new python code which writes the circuit in terms of the allowed Clifford generators $H$, $S$ and $CX$, allowed to be input to the compilation.

In the form produced by the cells above, the circuits will not be valid inputs for the compilation code.
So we will need to do that pre-processing beforehand.

To simplify this task, we made sure that the oracles are immediately written in terms of the desired Clifford generators ($Z=S\cdot S$ and $CZ= (I\otimes H) CX_{12} (I \otimes H)$), such that we will **only need to worry with the decomposition of the Toffoli gate**. This makes the processing of the `.qasm` file simpler as we need only work through that decomposotion.

In [ ]:
nr_samples = 100

create_path (n_ccz, n, s_string)

sample_gen(nr_samples, n, n_ccz, s, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n=14$


In [ ]:
n = 14

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 1 # implies t_count = 14
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 10

In [ ]:
create_path (n_ccz, n, s_string)

sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n=18$


In [ ]:
n = 18

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 1 # implies t_count = 14
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 10

In [ ]:
create_path (n_ccz, n, s_string)

sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n=22$


In [ ]:
n = 22

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 1 # implies t_count = 14
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 10

In [ ]:
create_path (n_ccz, n, s_string)

sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n=28$


In [ ]:
n = 28

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 1 # implies t_count = 14
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 2

In [ ]:
create_path (n_ccz, n, s_string)

sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n=32$


In [ ]:
n = 32

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 1 # implies t_count = 14
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 2

In [ ]:
create_path (n_ccz, n, s_string)

sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

## $t=28$

### $n=14$

In [ ]:
n = 14

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 2 # implies t_count = 28
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 2

In [ ]:
create_path (n_ccz, n, s_string)

sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n= 18$

In [ ]:
n = 18

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 2 # implies t_count = 28
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 2

In [ ]:
create_path (n_ccz, n, s_string)

sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n= 22$

In [ ]:
n = 22

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 2 # implies t_count = 28
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 2

In [ ]:
create_path (n_ccz, n, s_string)

sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n= 28$

In [ ]:
n = 28

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 2 # implies t_count = 28
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 2

In [ ]:
create_path (n_ccz, n, s_string)

sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n= 32$

In [ ]:
n = 32

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 2 # implies t_count = 28
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 2

In [ ]:
create_path (n_ccz, n, s_string)

sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n= 36$

In [ ]:
n = 36

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 2 # implies t_count = 28
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 1

In [ ]:
create_path (n_ccz, n, s_string)

sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

---

## Importing some circuits to evaluate the depths:


In [ ]:
#without Toffoli decomp
samples = [f'tcount28--n36/s000101000100100000010101110010110101/_samples/HSA{i}.qasm' for i in range(1)]

print ('\nWithout Toffoli decomp\n')
for i in range(1):
    imported_qc = qiskit.QuantumCircuit.from_qasm_file(samples[i])
    print(imported_qc.depth())
    
#with Toffoli decomp
samples = [f'tcount28--n36/s000101000100100000010101110010110101/_samples/HSA{i}-input.qasm' for i in range(1)]
print ('\nWith Toffoli decomp\n')
for i in range(1):
    imported_qc = qiskit.QuantumCircuit.from_qasm_file(samples[i])
    print(imported_qc.depth())



---